In [ ]:
# to import the necessary libraries

import pyActigraphy
import pandas as pd
import glob
import numpy as np
from scipy import ndimage
from pyActigraphy.metrics.metrics import _lmx
from pyActigraphy.sleep.scoring_base import _td_format


### TO EXTRACT THE MEAN 24-H AVERAGE WITH L5 AND M10

filenames = sorted(glob.glob('*.txt'))


# to read the raw data from each txt file

for f in filenames:
    rawATR = pyActigraphy.io.read_raw_atr(f)
    #rawATR = pyActigraphy.io.read_raw_atr(f)
    

# Just an example of variables we can extract. It calculates the variables and 
# print them in each original file to keep the link with file name
    
    raw = rawATR    
    
    lm5_ts, lm5 = _lmx(rawATR.resampled_data('10min'), '5h', lowest=True)
    lm10_ts, lm10 = _lmx(rawATR.resampled_data('10min'), '10h', lowest=False)
    
    if lm5_ts<pd.Timedelta('24h'):
        lm5_start = lm5_ts 
    else:
        lm5_start = lm5_ts - pd.Timedelta('24h')
        lm5_stop = lm5_start + pd.Timedelta('5h')
    
    if lm10_ts<pd.Timedelta('24h'):
        lm10_start = lm10_ts 
    else:
        lm10_start = lm10_ts - pd.Timedelta('24h')
        lm10_stop = lm10_start + pd.Timedelta('10h')
    
    daily_profile = raw.average_daily_activity(freq='15min', cyclic=False, binarize=False)

    # to prepare the figure

    layout = go.Layout(
        title="Daily activity profile",
        xaxis=dict(title="Date time"),
        shapes=[
            dict(
                type='rect',
                xref='x',
                yref='paper',
                x0=_td_format(lm5_start),
                y0=0,
                x1=_td_format(lm5_stop),
                y1=1,
                fillcolor='#ee4035',
                opacity=0.5,
                layer='below',
                line=dict(width=0)
            ),
            dict(
                type='rect',
                xref='x',
                yref='paper',
                x0=_td_format(lm10_start),
                y0=0,
                x1=_td_format(lm10_stop),
                y1=1,
                fillcolor='#7bc043',
                opacity=0.5,
                layer='below',
                line=dict(width=0)
            )
        ],
        showlegend=False);

    fig = go.Figure(data=[
        go.Scatter(x=daily_profile.index.astype(str), y=daily_profile)
    ], layout=layout)
    
    fig.write_image(f + "profile.png")
        

In [ ]:
filenames = sorted(glob.glob('*.txt'))


### to read the raw data from each txt file

for f in filenames:
    rawATR = pyActigraphy.io.read_raw_atr(f)
    #rawATR = pyActigraphy.io.read_raw_atr(f)
    

# Just an example of variables we can extract. It calculates the variables and 
# print them in each original file to keep the link with file name
    
    from plotly.graph_objs import Figure, FigureWidget, Layout, Scatter
    from plotly.offline import init_notebook_mode, iplot

    init_notebook_mode(connected=True)

    layout = Layout(
    title="",
    xaxis=dict(title="Date time"), 
    yaxis=dict(title="Activity/phase"), 
    showlegend=False
    )


    fig = go.Figure(data=[
        go.Scatter(x=raw.data.index.astype(str), y=raw.data)
    ], layout=layout)

    fig.write_image(f + "raw.png")